In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
ngpu= 1
print(torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu"))
print(torch.cuda.get_device_name(0))

from torchsummary import summary
import torch.nn as nn
from models.unet_models import *

1.8.1+cu101
True
cuda:0
NVIDIA GeForce RTX 2060


In [2]:
from complexPyTorch.complexLayers import *
from complexPyTorch.complexFunctions import *
from models.unet_parts import *
from models.unet_models import *
import torch.nn as nn
import numpy as np
class complexconv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, stride=1, groups=1):
        super(complexconv, self).__init__()
        self.conv = ComplexConv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding, stride=stride, groups=groups)
        self.bn = ComplexBatchNorm2d(out_channels)
        self.relu = ComplexReLU()
        self.leakyrelu = ComplexLeakyReLU()
        self.rrelu = ComplexRReLU()
        self.prelu = ComplexPReLU()
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        # x = self.leakyrelu(x)
        x = self.prelu(x)
        return x

class actlayer(nn.Module):
    def __init__(self):
        super(actlayer, self).__init__()
        self.relu = nn.LeakyReLU(inplace=True)
        self.tanh = nn.Tanh()
    def forward(self, x):
        a, p = x.chunk(2, 1)
        return torch.cat((self.relu(a), self.tanh(p)), dim=1)

class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, stride=1, groups=1):
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1, groups=groups)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.lrelu = nn.LeakyReLU(inplace=True)
        self.prelu = nn.PReLU()
        self.tanh = nn.Tanh()
        self.act = actlayer()
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class FUnet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(FUnet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        self.fft = FourierTransform_Coe()

        self.conv1 = complexconv(in_channels=3, out_channels=8, groups=1)
        self.conv2 = complexconv(in_channels=8, out_channels=8, groups=1)
        # self.inc = DoubleConv(n_channels, 8)

        self.down1 = Down(8, 16)
        self.down2 = Down(16, 32)
        self.down3 = Down(32, 64)
        factor = 2 if bilinear else 1
        self.down4 = Down(64, 128 // factor)
        self.up1 = Up(128, 64 // factor, bilinear)
        self.up2 = Up(64, 32 // factor, bilinear)
        self.up3 = Up(32, 16 // factor, bilinear)
        self.up4 = Up(16, 8, bilinear)
        self.outc = OutConv(8, n_classes)

    def forward(self, x):
        # x1 = self.inc(x)
        x = self.fft.transform(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x1 = self.fft.inverse(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [8]:
from complexPyTorch.complexLayers import *
from complexPyTorch.complexFunctions import *
from models.unet_parts import *
from models.unet_models import *
import torch.nn as nn
import numpy as np

class complexconv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, stride=1, groups=1):
        super(complexconv, self).__init__()
        self.conv = ComplexConv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding, stride=stride, groups=groups)
        self.bn = ComplexBatchNorm2d(out_channels)
        self.relu = ComplexReLU()
        self.leakyrelu = ComplexLeakyReLU()
        self.rrelu = ComplexRReLU()
        self.prelu = ComplexPReLU()
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        # x = self.leakyrelu(x)
        x = self.prelu(x)
        return x

class ffe(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ffe, self).__init__()
        self.n_channels = in_channels
        self.n_classes = out_channels
        self.fft = FourierTransform_Coe()
        self.conv = complexconv(in_channels=in_channels, out_channels=out_channels)
    def forward(self, x):
        x = self.fft.transform(x)
        x = self.conv(x)
        x = self.fft.inverse(x)
        return x

class ffn(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels):
        super(ffn,self).__init__()
        self.in_channels = in_channels
        self.mid_channels = mid_channels
        self.out_channels = out_channels
        self.fblock = ffe(in_channels=self.in_channels, out_channels=self.mid_channels)
        self.conv = nn.Conv2d(in_channels=self.mid_channels*3, out_channels=self.out_channels, kernel_size=1)
        self.down = nn.MaxPool2d(2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
    def forward(self, x):
        c1 = self.fblock(x)

        x2 = self.down(x)
        c2 = self.fblock(x2)
        c2 = self.up(c2)

        x3 = self.down(x2)
        c3 = self.fblock(x3)
        c3 = self.up(c3)
        c3 = self.up(c3)

        # x4 = self.down(x3)
        # c4 = self.fblock(x4)
        # c4 = self.up(c4)
        # c4 = self.up(c4)
        # c4 = self.up(c4)

        c = torch.cat([c1, c2, c3], 1)
        x = self.conv(c)
        return x
class fcn(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(fcn, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.ffn = ffn(in_channels=3, mid_channels=8, out_channels=n_channels)
        self.tini_unet = tini_unet(n_channels=n_channels, n_classes=n_classes)

    def forward(self, x):
        saliency_map = self.ffn(x)
        logits = self.tini_unet(x)
        return logits, saliency_map

model = fcn(3, 2).cuda()
summary(model, (3, 224, 224), dtypes=[torch.float])

Layer (type:depth-idx)                        Output Shape              Param #
├─ffn: 1-1                                    [-1, 3, 224, 224]         --
|    └─ffe: 2-1                               [-1, 8, 224, 224]         --
|    |    └─complexconv: 3-1                  [-1, 8, 224, 224]         489
|    └─MaxPool2d: 2-2                         [-1, 3, 112, 112]         --
|    └─ffe: 2-3                               [-1, 8, 112, 112]         (recursive)
|    |    └─complexconv: 3-2                  [-1, 8, 112, 112]         (recursive)
|    └─Upsample: 2-4                          [-1, 8, 224, 224]         --
|    └─MaxPool2d: 2-5                         [-1, 3, 56, 56]           --
|    └─ffe: 2-6                               [-1, 8, 56, 56]           (recursive)
|    |    └─complexconv: 3-3                  [-1, 8, 56, 56]           (recursive)
|    └─Upsample: 2-7                          [-1, 8, 112, 112]         --
|    └─Upsample: 2-8                          [-1, 8, 224,

Layer (type:depth-idx)                        Output Shape              Param #
├─ffn: 1-1                                    [-1, 3, 224, 224]         --
|    └─ffe: 2-1                               [-1, 8, 224, 224]         --
|    |    └─complexconv: 3-1                  [-1, 8, 224, 224]         489
|    └─MaxPool2d: 2-2                         [-1, 3, 112, 112]         --
|    └─ffe: 2-3                               [-1, 8, 112, 112]         (recursive)
|    |    └─complexconv: 3-2                  [-1, 8, 112, 112]         (recursive)
|    └─Upsample: 2-4                          [-1, 8, 224, 224]         --
|    └─MaxPool2d: 2-5                         [-1, 3, 56, 56]           --
|    └─ffe: 2-6                               [-1, 8, 56, 56]           (recursive)
|    |    └─complexconv: 3-3                  [-1, 8, 56, 56]           (recursive)
|    └─Upsample: 2-7                          [-1, 8, 112, 112]         --
|    └─Upsample: 2-8                          [-1, 8, 224,

## model summary

<p>繪製 Model:<p>

<pre><code>from torchviz import make_dot
x = torch.randn(1, 3, 256, 256).requires_grad_(True).cuda()
y = model(x)
vis_graph = make_dot(y, params=dict(list(model.named_parameters()) + [('x', x)]))
vis_graph.view()
</code></pre>

<p>輸出 Model parameters:
<pre><code> 
model = UNet(n_class=3).cuda()
for name, param in model.named_parameters():
     print(name, param.shape)
</code></pre>

<pre><code>
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)
</code></pre>

## training loop

In [ ]:
from dataloader import *
import torchvision
from torchvision import transforms, datasets, models
import os
from torch.utils.data import DataLoader, random_split

tr_img_folder = os.path.join(r'C:\Users\user\pythonProject\mission87\data\DUTS\DUTS-TR\DUTS-TR-Image')
tr_gt_folder = os.path.join(r'C:\Users\user\pythonProject\mission87\data\DUTS\DUTS-TR\DUTS-TR-Mask')

val_percent = 0.0
batch_size = 8

train_set = DatasetGenerate(tr_img_folder, tr_gt_folder, phase='train', test_size=0.1)
val_set = DatasetGenerate(tr_img_folder, tr_gt_folder, phase='val', test_size=0.1)

loader_args = dict(batch_size=batch_size, num_workers=4, prefetch_factor =2 , pin_memory=True)
train_loader = DataLoader(train_set, shuffle=True, **loader_args)
val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)

dataloaders = {
    'train': train_loader,
    'val': val_loader
}

from itertools import chain
optimizer = torch.optim.Adam(params=chain(model.ffn.parameters(),
                                          model.tini_unet.parameters()),
                            lr=1e-3)

### MSRA10K

In [ ]:
# from dataloader import *
# import torchvision
# from torchvision import transforms, datasets, models
# import os
# from torch.utils.data import DataLoader, random_split
# # MSRA10K 沒有預先分類 train, test dataset 因此多一步處理
# tr_img_folder = os.path.join(r'C:\Users\User\pythonProject\mission87\data\MSRA10K_Imgs_GT\Images')
# tr_gt_folder = os.path.join(r'C:\Users\User\pythonProject\mission87\data\MSRA10K_Imgs_GT\Mask')

# val_percent = 0.1
# batch_size = 8

# tr_set = DatasetGenerate(tr_img_folder, tr_gt_folder, phase='train', test_size=0.1)
# ts_set = DatasetGenerate(tr_img_folder, tr_gt_folder, phase='val', test_size=0.1)
# n_val = int(len(tr_set) * val_percent)
# n_train = len(tr_set) - n_val
# train_set, val_set = random_split(tr_set, [n_train, n_val], generator=torch.Generator().manual_seed(0)) # MSRA10K: [train_set, test_set], DUTS: [train_set, val_set]

# loader_args = dict(batch_size=batch_size, num_workers=4, prefetch_factor =2 , pin_memory=True)
# train_loader = DataLoader(train_set, shuffle=True, **loader_args)
# val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)

# dataloaders = {
#     'train': train_loader,
#     'val': val_loader
# }

# Start Training

In [6]:
'''
model : {unet, complexunet, amlitude_phase}
phase: {}
lossfunction: {MSE, FFL, BCE, BCE-Dice}
'''
from train import train_model
# model = testunet16(n_channels=8, n_classes=3)
# model = amlitude_phase(3, 3)
# model = fcn(3, 2)
# summary(model, (1, 224, 224), dtypes=[torch.float])
model = train_model(model=model, dataloaders=dataloaders, optimizer=optimizer, num_epochs=50, phase='spatial domain', task='FFNv2', lossfunction='BCE-Dice')

# Test loss func. & dataset

## test loss function

In [ ]:
from models.unet_models import *
model = UNet(n_channels=3, n_classes=3).cuda()
# summary(model, (3, 224, 224), dtypes=[torch.float])

criterion = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch = next(iter(dataloaders['train']))
image = batch['image'].to(device=device, dtype=torch.float32)
mask = batch['mask'].to(device=device, dtype=torch.long)

pred = model(image.to(torch.float))

criterion(pred, mask[:,-1,:,:])

In [ ]:
mask.shape

In [ ]:
criterion = BceDiceLoss(multiclass=True, n_classes=model.n_classes)
criterion(pred, mask)

In [ ]:
def dice_loss(pred, target, smooth=1.):
    """Dice loss
    """
    pred = pred.contiguous()
    target = target.contiguous()

    intersection = (pred * target).sum(dim=2).sum(dim=2)

    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))

    return loss.mean()


class Weighted_Cross_Entropy_Loss(torch.nn.Module):
    """Cross entropy loss that uses weight maps."""

    def __init__(self):
        super(Weighted_Cross_Entropy_Loss, self).__init__()

    def forward(self, pred, target, weights=10):
        n, c, H, W = pred.shape
        # # Calculate log probabilities
        logp = F.log_softmax(pred, dim=1)

        # Gather log probabilities with respect to target
        logp = torch.gather(logp, 1, target.view(n, 1, H, W))

        # Multiply with weights
        weighted_logp = (logp * weights).view(n, -1)

        # Rescale so that loss is in approx. same interval
        weighted_loss = weighted_logp.sum(1) / weights.view(n, -1).sum(1)

        # Average over mini-batch
        weighted_loss = -weighted_loss.mean()

        return weighted_loss

In [ ]:
criterion = Weighted_Cross_Entropy_Loss()
loss = criterion(pred, mask)

In [ ]:
loss = torch.nn.BCELoss()
loss(F.sigmoid(pred), mask)

In [ ]:
softmax = torch.nn.Softmax(dim=1)
softmax(pred).shape

In [ ]:
test = torch.randn([1, 3, 8, 8], requires_grad=True)
test = test.to(torch.cfloat) #dtype to complex64
test = test.cuda()
# print(test)
# print(test)

pred = torch.ones([1, 3, 8, 8], requires_grad=True)
pred = pred.to(torch.cfloat) #dtype to complex64
pred = pred.cuda()
# print(pred)

mse = MSELoss() #因實部, 虛部拆分計算, 因此 n 會 *2
loss = mse(pred, test)
print('custom mse function', loss)
loss.backward()

print('torch native mse funciton', torch.nn.functional.mse_loss(pred.to(torch.float), test.to(torch.float)))

In [ ]:
pred_freq = torch.stack([pred.real, pred.imag], -1)
target_freq = torch.stack([test.real, test.imag], -1)
print(pred_freq, target_freq, pred_freq.shape)

In [ ]:
freq_distance = (pred_freq - target_freq) ** 2
print(freq_distance)

In [ ]:
torch.mean(freq_distance)

## check training/valid data

In [ ]:
import matplotlib.pyplot as plt
from utils.utils import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch = next(iter(dataloaders['train']))
image = batch['image']
mask = batch['mask']

s = {0: 'orignal image & mask',1: 'Fourier coefficients (with mask)', 2: 'Fourier coefficients (with image*mask)', 3: 'Fourier amplitude', 4: 'Fourier phase'}
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
fig.suptitle('orignal image & mask', verticalalignment='bottom')
ax1.imshow(image[0].permute(1, 2, 0), 'gray')
ax1.set_title('image')
ax2.imshow(mask[0].permute(1, 2, 0)*255, 'gray')
ax2.set_title('mask')
fig.tight_layout()
fig.subplots_adjust(top=0.99)
plt.show()

In [ ]:
o = model(image.cuda())

In [ ]:
plt.imshow(o[0].permute(1, 2, 0).cpu())

In [ ]:
from models.unet_parts import FFT2
f = FFT2()
am, p = f.transform(image[0])
i = f.inverse(am, p)

In [ ]:
plt.imshow(torch.log(am).permute(1, 2, 0))

In [ ]:
plt.imshow(i.permute(1, 2, 0))

In [ ]:
f = np.fft.fft2(image[0])
fshift = np.fft.fftshift(f)
amplitude = np.absolute(fshift)
phase = np.angle(fshift)

ishift = np.fft.ifftshift(amplitude*np.exp(1j*phase))
iimg = np.fft.ifft2(ishift)
iimg = np.absolute(iimg)

In [ ]:
image[0] = image[0] / 255

In [ ]:
plt.imshow(np.log(amplitude).transpose(1, 2, 0))

In [ ]:
plt.imshow(iimg.permute(1, 2, 0))

# Eval

In [ ]:
## test_unet eval
import matplotlib.pyplot as plt
from utils.utils import *
from PIL import Image
# model = torch.load(r'C:\Users\User\pytho/nProject\mission87\task\BCE unet\net\net-50.pkl').cuda()
# model = torch.load(r'C:\Users\User\pythonProject\mission87\task\fcn-with-coe-v2\net\best.pkl')
# model.load_state_dict(torch.load(r'C:\Users\User\pythonProject\mission87\task\fcn-with-coe-v3\net\best.pkl'))
'''
load model from task
'''
# model = torch.load(r'C:\Users\User\pythonProject\mission87\task\funet\net.pkl')
# model.load_state_dict(torch.load(r'C:\Users\User\pythonProject\mission87\task\funet\net\best.pkl'))


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.eval()
with torch.no_grad():
    batch = next(iter(dataloaders['val']))
    image = batch['image'].to(device=device, dtype=torch.float32)
    mask = batch['mask'].to(device=device, dtype=torch.long)[:, -1, :, :]
    pred = model(image)
    for i in range(batch_size):
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(15, 5))
        ax1.imshow((image[i]).permute(1, 2, 0).cpu().numpy())
        ax1.set_title('image')
        ax2.imshow(mask[i].cpu())
        ax2.set_title('mask')
        # ax3.imshow((pred[i]).permute(1, 2, 0).cpu().numpy(), 'gray') #BCE 沒有
        # ax3.set_title('pred')
        ax4.imshow(binary(pred[i], model.n_classes))
        # ax4.imshow(objectmap(pred[i]))
        ax4.set_title('binary pred')
        plt.show()

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
model.conv2.register_forward_hook(get_activation('fc3'))
output = model(image)
mid_output = activation['fc3']

m_out = mid_output[0]
f = FourierTransform_Coe()
img = f.inverse(m_out) *255.0
plt.imshow(img.permute(1, 2, 0).cpu())

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(15, 5))
i = 7
# ax1.imshow(pred[i].permute(1, 2, 0).cpu().numpy())
ax1.set_title('pred')
ax2.imshow(pred[i][0].cpu())
ax2.set_title('pred[0]')
ax3.imshow(pred[i][1].cpu()) #BCE 沒有
ax3.set_title('pred[1]')
ax4.imshow(pred[i][2].cpu())
ax4.set_title('pred[2]')
plt.show()

In [ ]:
plt.imshow(image[5].permute(1, 2, 0).cpu())
plt.axis('off')

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
i = 5
ax1.imshow(pred[i][0].cpu()) #BCE 沒有
ax1.set_title('pred[1]')
ax2.imshow(pred[i][1].cpu())
ax2.set_title('pred[2]')
plt.show()

In [ ]:
plt.imshow(pred[5][1].cpu())
plt.axis('off')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torchviz import make_dot
batch = next(iter(dataloaders['train']))
image = batch['image'].to(device=device, dtype=torch.float32)
mask = batch['mask'].to(device=device, dtype=torch.long)[:, -1, :, :]
pred = model(image)
# yhat = model(batch.text) # Give dummy batch to forward().
make_dot(pred, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

In [ ]:
img = pred[3]
plt.hist(img.ravel().cpu().numpy(), bins=50, density=True)
plt.xlabel("pixel values")
plt.ylabel("relative frequency")
plt.title("distribution of pixels")

In [ ]:
p = Image.open(r'C:\Users\User\pythonProject\mission87\data\DUTS\DUTS-TR\DUTS-TR-Image\ILSVRC2012_test_00000004.jpg')
p = np.array(p) / 255.0
p = torch.as_tensor(p.copy()).float().contiguous().permute(2, 1, 0)
p = p.unsqueeze(0)

In [ ]:
p.shape

In [ ]:
pool = torch.nn.Conv2d(in_channels=3, out_channels=3, kernel_size=2, padding=0, stride=2)

In [ ]:
fft = FFT()
p_coe = fft.transform(p)
p2 = fft.inverse(p_coe)

In [ ]:
img = p2
img -= torch.min(img)
img /= torch.max(img)
img

In [ ]:
p = pred
p -= torch.min(p)
p /= torch.max(p)
plt.imshow(p[0].permute(1, 2, 0).cpu())

## 其他檢查

In [ ]:
# channels=3
pred -= torch.min(pred)
pred /= torch.max(pred)
pred

probs = torch.sigmoid(pred[0])[0]

In [ ]:
pred -= torch.min(pred)
pred /= torch.max(pred)
pred
# plot the pixel values
plt.hist(pred[0].cpu().numpy().ravel(), bins=50, density=True)
plt.xlabel("pixel values")
plt.ylabel("relative frequency")
plt.title("distribution of pixels")

In [ ]:
full_mask = pred[0].cpu().squeeze()
mask = ((full_mask > 0.5).float()*1).numpy()
result = Image.fromarray((mask * 255).astype(np.uint8).transpose(1, 2, 0))
result

In [ ]:
import torch.nn.functional as F
probs = torch.sigmoid(pred[2].unsqueeze(0))[0]
full_mask = probs.cpu().squeeze()

mask = F.one_hot(full_mask.argmax(dim=0), 3).permute(2, 0, 1).numpy()

result = Image.fromarray((np.argmax(mask, axis=0) * 255 / mask.shape[0]).astype(np.uint8))
result

# Eval: test dataset

In [ ]:
from dataloader import *
import torchvision
from torchvision import transforms, datasets, models
import os
from torch.utils.data import DataLoader, random_split
from utils.utils import *
te_img_folder = os.path.join(r'C:\Users\user\pythonProject\mission87\data\DUTS\DUTS-TE\DUTS-TE-Image')
te_gt_folder = os.path.join(r'C:\Users\user\pythonProject\mission87\data\DUTS\DUTS-TE\DUTS-TE-Mask')

batch_size = 4

dataset = DatasetGenerate(te_img_folder, te_gt_folder)

loader_args = dict(batch_size=batch_size, num_workers=4, prefetch_factor =2 , pin_memory=True)
test_loader = DataLoader(dataset, shuffle=False, **loader_args)

dataloaders = {
    'test': test_loader
}

from models.unet_models import *
# model = torch.load(r'C:\Users\User\pythonProject\mission87\task\BCE unet\net\net-50.pkl')
model.load_state_dict(torch.load(r'C:\Users\User\pythonProject\mission87\task\FFN\net\best.pkl'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
model.eval()
with torch.no_grad():
    batch= next(iter(dataloaders['test']))
    image = batch['image'].to(device=device, dtype=torch.float32)
    mask = batch['mask'].to(device=device, dtype=torch.long)[:, -1, :, :]
    pred = model(image)
    for i in range(batch_size):
        fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
        ax1.imshow((image[i]).permute(1, 2, 0).cpu().numpy())
        ax1.set_title('image')
        ax2.imshow(mask[i].cpu())
        ax2.set_title('mask')
        ax3.imshow(binary(pred[i], model.n_classes))
        # ax3.imshow(objectmap(pred[i]))
        ax3.set_title('binary pred')
        plt.show()

In [ ]:
features_in_hook = []
features_out_hook = []

def hook(module, fea_in, fea_out):
    features_in_hook.append(fea_in)
    features_out_hook.append(fea_out.detach().cpu().numpy())
    return None

model._modules.get('conv').register_forward_hook(hook=hook)
result = model(image)
print(features_in_hook)  # 勾的是指定层的输入
print(features_out_hook)  # 勾的是指定层的输出

In [ ]:
from itertools import chain
torch.optim.Adam(params=chain(model.fblock.parameters(),
                        model.conv.parameters(),
                        model.unet.parameters()),lr=1e-3)

In [ ]:
out, features_in_forward, features_out_forward = model(image)
print("*"*5+"forward return features"+"*"*5)
print(features_in_forward)
print(features_out_forward)
print("*"*5+"forward return features"+"*"*5)

In [ ]:
model.named_modules('conv')

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
i = 3
ax1.imshow(pred[i][0].cpu()) #BCE 沒有
ax1.set_title('pred[1]')
ax2.imshow(pred[i][1].cpu())
ax2.set_title('pred[2]')
plt.show()

In [ ]:
pred[i][0], pred[i][1]

In [ ]:
pred_output = torch.tensor([[[[0.3157, 0.7023], [0.7108, 0.3082]], [[0.6998, 0.3112], [0.3067, 0.7068]]]])
print(pred_output)
temp1 = F.softmax(pred_output,dim=1)
print("temp1:",temp1)

temp3 = torch.log(temp1)
print("temp3:",temp3)

target = torch.tensor([[[1,0],[0,1]]])
target = target.long()
loss1 = F.nll_loss(temp3,target)
print('loss1: ', loss1)

In [ ]:
o = binary(pred_output, model.n_classes)
print(o)

# save output

In [ ]:
from torchvision.utils import save_image
from dataloader import *
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# te_img_folder = os.path.join(r'C:\Users\user\pythonProject\mission87\data\DUTS\DUTS-TE\DUTS-TE-Image')
# te_gt_folder = os.path.join(r'C:\Users\user\pythonProject\mission87\data\DUTS\DUTS-TE\DUTS-TE-Mask')
te_img_folder = os.path.join(r'C:\Users\User\pythonProject\mission87\eval_outputs\msra5')
te_gt_folder = os.path.join(r'C:\Users\User\pythonProject\mission87\eval_outputs\mask-msra55')
batch_size = 1

dataset = DatasetGenerate(te_img_folder, te_gt_folder, test_size=0.0)

loader_args = dict(batch_size=batch_size, num_workers=4, prefetch_factor =2 , pin_memory=True)
loader_args = dict(batch_size=batch_size)
test_loader = DataLoader(dataset, shuffle=False, **loader_args)

dataloaders = {
    'test': test_loader
}


root_path = r'C:\Users\User\pythonProject\mission87\eval_outputs'
pred_path = r'unet-msra'
mask_path = r'mask-msra'

p = pred_path
# for p in p:
if not os.path.exists(os.path.join(root_path, p)):
    os.makedirs(os.path.join(root_path, p))

In [ ]:
model.load_state_dict(torch.load(r'C:\Users\User\pythonProject\mission87\task\unet-MSRA10K\net\best.pkl'))

In [ ]:
from utils.utils import *
# model = torch.load(r'C:\Users\User\pythonProject\mission87\task\BCE unet\net\net-50.pkl').cuda()
# model = torch.load(r'C:\Users\User\pythonProject\mission87\task\BCE unet\net\net-3.pkl')
# image_path = r'images'
i = 0
with torch.no_grad():
    for batch in tqdm(dataloaders['test']):
        image = batch['image'].to(device=device, dtype=torch.float32)
        mask = batch['mask'].to(device=device, dtype=torch.long)[:, -1, :, :]
        pred = model(image)
        pred__path = os.path.join(root_path, pred_path, str(i) + '.png')
        Image_copy = Image.Image.copy(binary(pred[0], model.n_classes))
        # Image_copy = Image.Image.copy(objectmap(pred[0]))
        Image.Image.save(Image_copy, pred__path)
        save_image(mask[0,:,:].to(device=device, dtype=torch.float), os.path.join(root_path, 'mask-msra', str(i) + '.png'))
        i += 1

In [13]:
# model = torch.load(r'C:\Users\User\pythonProject\mission87\task\BCE unet\net\net-50.pkl').cuda()
device = torch.device('cuda')
model.to(device=device)
repetitions = 300
timings=np.zeros((repetitions,1))
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
#GPU-WARM-UP
dummy_input = torch.randn(1, 3, 224, 224,dtype=torch.float).to(device)
for _ in range(100):
   _ = model(dummy_input)

# MEASURE PERFORMANCE
with torch.no_grad():
   for rep in range(repetitions):
      starter.record()
      _ = model(dummy_input)
      ender.record()
      # WAIT FOR GPU SYNC
      torch.cuda.synchronize()
      curr_time = starter.elapsed_time(ender)
      timings[rep] = curr_time

mean_syn = np.sum(timings) / repetitions
std_syn = np.std(timings)
mean_fps = 1000. / mean_syn
print('Mean: {mean_syn:.3f}ms Std: {std_syn:.3f}ms FPS: {mean_fps:.2f}'.format(mean_syn=mean_syn, std_syn=std_syn, mean_fps=mean_fps))
print(mean_syn)

Mean: 20.429ms Std: 0.588ms FPS: 48.95
20.42941535949707


In [ ]:
import time
model = torch.load(r'C:\Users\User\pythonProject\mission87\task\BCE unet\net\net-50.pkl')
device = torch.device('cpu')
model.to(device=device)
repetitions = 300
timings=np.zeros((repetitions,1))
dummy_input = torch.randn(1, 3, 224, 224,dtype=torch.float).to(device)
with torch.no_grad():
   for rep in range(repetitions):
      starter = time.perf_counter()
      _ = model(dummy_input)
      curr_time = time.perf_counter() - starter
      timings[rep] = curr_time

mean_syn = np.sum(timings) / repetitions
std_syn = np.std(timings)
mean_fps = 1. / mean_syn
print('Mean: {mean_syn:.3f}s Std: {std_syn:.3f}s FPS: {mean_fps:.2f}'.format(mean_syn=mean_syn, std_syn=std_syn, mean_fps=mean_fps))
print(mean_syn)

In [ ]:
# test_loader = get_loader(te_img_folder, te_gt_folder, phase='test',
#                           batch_size=8, shuffle=True, num_workers=4, prefetch_factor=2,
#                           transform=test_transform)

# with torch.no_grad():
#     image, mask = next(iter(test_loader))
#     image, mask = image.to(torch.float).cuda(), mask.to(torch.float).cuda()
#     pred = model_1(image)
#     pred2 = model_2(image)
#     for i in range(8):
#         image_ = (image[i]).permute(1, 2, 0).cpu().numpy()
#         mask_ = (mask[i]).permute(1, 2, 0).cpu().numpy()
#         pred_ = (pred[i]).permute(1, 2, 0).cpu().detach().numpy()
#         pred_2 = (pred2[i]).permute(1, 2, 0).cpu().detach().numpy()

#         fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(15, 5))
#         ax1.imshow(image_)
#         ax1.set_title('image')
#         ax2.imshow(mask_)
#         ax2.set_title('mask')
#         ax3.imshow(pred_)
#         ax3.set_title('pred by orig')
#         ax4.imshow(pred_2)
#         ax4.set_title('pred by freq')
#         plt.show()